# RAG Architecture Sample in Python (Azure AI Search)
このサンプルノートブックでは以下の RAG アーキテクチャの動作を試すことができます。


# 事前準備
この Python サンプルを実行するには、以下が必要です：
- Azure AI Search リソース。エンドポイントとクエリ API キーが必要です。
- Azure OpenAI Service にアクセスできる承認済み Azure サブスクリプション
- Azure OpenAI Service への `text-embedding-ada-002` Embeddings モデルのデプロイメント。このデモでは、API バージョン `2023-05-15` を使用しています。デプロイ名はモデルと同じ「`text-embedding-ada-002`」を使用しています。
- Azure OpenAI Service の接続とモデル情報
  - OpenAI API キー
  - OpenAI Embeddings モデルのデプロイメント名
  - OpenAI API バージョン
- Python (この手順はバージョン 3.10.x でテストされています)

これらのデモには、Visual Studio Code と [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) を使用できます。

## パッケージのインストール

In [ ]:
!pip install azure-search-documents==11.4.0
!pip install openai

In [ ]:
import azure.search.documents
azure.search.documents.__version__

## 必要なライブラリと環境変数のインポート

In [ ]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient, SearchIndexingBufferedSender  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryCaptionResult,
    QueryAnswerResult,
    SemanticErrorMode,
    SemanticErrorReason,
    SemanticSearchResultsType,
    QueryType,
    VectorizedQuery,
    VectorQuery,
    VectorFilterMode,    
)

## Azure AI Search 接続設定

In [ ]:
service_endpoint: str = "<Your search service endpoint>"
service_query_key: str = "<Your search service query key>"
index_name: str = "gptkbindex" #自動構築時のデフォルト設定

credential = AzureKeyCredential(service_query_key)

## Azure OpenAI の設定

In [ ]:
AZURE_OPENAI_API_KEY = "Your OpenAI API Key"
AZURE_OPENAI_ENDPOINT = "https://<Your OpenAI Service>.openai.azure.com/"
AZURE_OPENAI_CHATGPT_DEPLOYMENT = "chat" #自動構築時のデフォルト設定
AZURE_OPENAI_EMB_DEPLOYMENT="embedding" #自動構築時のデフォルト設定

In [ ]:
from openai import AzureOpenAI
from openai.types.chat import (
    ChatCompletion,
    ChatCompletionChunk,
)

from tenacity import retry, wait_random_exponential, stop_after_attempt  

openai_client = AzureOpenAI(
  api_key = AZURE_OPENAI_API_KEY,  
  api_version = "2023-05-15",
  azure_endpoint = AZURE_OPENAI_ENDPOINT
)

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# タイトルフィールドとコンテンツフィールドのEmbeddingsを生成する関数。
def generate_embeddings(text, model=AZURE_OPENAI_EMB_DEPLOYMENT):
    return openai_client.embeddings.create(input = [text], model=model).data[0].embedding

# 1. 検索クエリ生成
最新の質問とチャット履歴をもとに GPT-3.5 Turbo モデルを利用
したプロンプトエンジニアリングによって検索クエリを生成します。検索クエリーのフォーマットを合わせるために、Few-shot サンプルを用意して精度を高めています。

## 1.1. システムメッセージの設定

In [ ]:
# Query generation prompt
query_prompt_template = """
以下は、過去の会話の履歴と、日本史に関するナレッジベースを検索して回答する必要のあるユーザーからの新しい質問です。
会話と新しい質問に基づいて、検索クエリを作成してください。
検索クエリには、引用されたファイルや文書の名前（例:info.txtやdoc.pdf）を含めないでください。
検索クエリには、括弧 []または<<>>内のテキストを含めないでください。
検索クエリを生成できない場合は、数字 0 だけを返してください。
"""

messages = [{'role': 'system', 'content': query_prompt_template}]

## 1.2. Few-shot サンプルの設定

In [ ]:
# Few-shot Samples
query_prompt_few_shots = [
    {'role' : 'user', 'content' : '徳川家康ってなにした人  ' },
    {'role' : 'assistant', 'content' : '徳川家康 人物 歴史' },
    {'role' : 'user', 'content' : '徳川家康の武功を教えてください' },
    {'role' : 'assistant', 'content' : '徳川家康 人物 武功 業績' }
]

for shot in query_prompt_few_shots:
    messages.append({'role': shot.get('role'), 'content': shot.get('content')})

## 1.3. ユーザーからの質問

In [ ]:
# User query
user_q = "源実朝ってどんな人"
messages.append({'role': 'user', 'content': user_q})

## 1.4. 送信するメッセージの確認

In [ ]:
messages

## 1.5. 検索クエリ生成

In [ ]:
chat_completion: ChatCompletion = openai_client.chat.completions.create(
    messages=messages,
    model=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    temperature=0.0,
    max_tokens=100,
    n=1)

query_text = chat_completion.choices[0].message.content
print(query_text)

# 2. 検索インデックスから関連文書を取得（Retrieve）
1. で生成した検索クエリを使用して Azure AI Search で検索を行います。このサンプルでは検索クエリとベクトルの組み合わせでハイブリッド検索を行います。

In [ ]:
def nonewlines(s: str) -> str:
    return s.replace('\n', ' ').replace('\r', ' ').replace('[', '【').replace(']', '】')

In [ ]:
search_client = SearchClient(service_endpoint, index_name, credential=credential)
docs = search_client.search(
    search_text=query_text,
    filter=None,
    top=3,
    vector_queries=[VectorizedQuery(vector=generate_embeddings(query_text), k_nearest_neighbors=3, fields="embedding")]
)

In [ ]:
results =[" SOURCE:" + doc['sourcepage'] + ": " + nonewlines(doc['content']) for doc in docs]
print(results)

# 3. ChatGPT を利用した回答の生成

Azure AI Search の検索結果やチャット履歴を利用して、コンテキストや内容に応じた回答をを生成します。ここでプロンプトを使って出典を出力するように指示しています。出典には Azure AI Search のファイル名のフィールドの値を使用します。

システムメッセージは精度を高めるため、一部英語で記述しています。

In [ ]:
# System message
system_message_chat_conversation = """
日本の鎌倉時代の歴史に関する読解問題に答えるアシスタントです。
If you cannot guess the answer to a question from the SOURCE, answer "I don't know".
Answers must be in Japanese.

# Restrictions
- The SOURCE prefix has a colon and actual information after the filename, and each fact used in the response must include the name of the source.
- To reference a source, use a square bracket. For example, [info1.txt]. Do not combine sources, but list each source separately. For example, [info1.txt][info2.pdf].
"""

messages = [{'role': 'system', 'content': system_message_chat_conversation}]

## 3.1. コンテキストを拡張（Augument）

In [ ]:
# User query
user_q = "源実朝ってどんな人"
# Context from Azure AI Search
context = "\n".join(results)
messages.append({'role': 'user', 'content': user_q + "\n\n" + context}) 

## 3.2. 送信するメッセージの確認

In [ ]:
messages

## 3.3. 回答を生成（Generation）

In [ ]:
# ChatCompletion で回答を生成する
chat_coroutine = openai_client.chat.completions.create(
    model=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    messages=messages,
    temperature=0.0,
    max_tokens=1024,
    n=1,
    stream=False
)

print(chat_coroutine.choices[0].message.content)